# Imports

In [1]:
from IPython.display import display as dsp
from datetime import datetime as dt
from random import sample


import sys
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
from canalyst_candas.configuration.config import Config

config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "/home/canalyst-candas/canalyst",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



A configuration file has been created for you in 
/home/canalyst-candas/canalyst/keys.json. For Excel file downloads and scenario 
mapping, you'll need to add an API key to the 'canalyst_api_key' 
value. Visit https://app.canalyst.com/settings/api-tokens to create 
and retrieve your API key. You will also need to retrieve your 
S3 Access ID and Secret Key and fill in 'canalyst_s3_id' and 
'canalyst_s3_key' with those values, respectively. If using a 
Jupyter Notebook, stop and restart the notebook for the changes to 
take effect. If using a Python/iPython session, quit the current 
session and start a new one.


# Creating A Search Object

In [2]:
# Instantiate new search object:
search = cd.Search(config)


# Searching for Consumer Companies with Same Store Sales

In [3]:
search_results_df = search.search_time_series(
    sector="Consumer",
    time_series_name="sss",
    is_driver=True   
)

Building search index...
No.of.unique tickers:  114
No.of.unique time series:  96


In [4]:
# Common time series:
gpd_search = search_results_df.groupby("time_series_name")[["ticker"]].nunique().reset_index().copy()
gpd_search.sort_values(by="ticker", ascending=False, inplace=True)
gpd_search.head(10)

,time_series_name,ticker
18,MO_OS_SSS_NewStoreEfficiency,58
1,MO_GA_SSS,44
11,MO_OS_SSS_53rdWeek,43
21,MO_OS_SSS_OtherImpact,14
10,MO_GA_SSS_Traffic,12
14,MO_OS_SSS_ClosedStoreEfficiency,11
2,MO_GA_SSS_AverageCheck,8
19,MO_OS_SSS_NewStoreEfficiency_CompanyOwned,7
6,MO_GA_SSS_Franchised,5
4,MO_GA_SSS_CompanyOwned,4


In [5]:
ts = "MO_GA_SSS"
search_results_df = search.search_time_series(
    sector="Consumer",
    time_series_name="MO_GA_SSS",
    unit_type='percentage',
    is_driver=True
)

No.of.unique tickers:  64
No.of.unique time series:  10


In [6]:
# A list of tickers for this driver:
store_tickers = list(search_results_df[search_results_df["time_series_name"] == ts]["ticker"].unique())
len(store_tickers)

43

# Create a Model Set of "SSS Companies"

In [7]:
# Create a model set form this ticker list:
start = dt.now()
store_set = cd.ModelSet(ticker_list=store_tickers, config=config, allow_nulls=True)
dsp((dt.now() - start).seconds)

Please instantiate ModelSet with a list of tickers, even a list of one.


70

In [8]:
start = dt.now()
# Create params:
df_params = store_set.forecast_frame(ts,
                             n_periods=range(0,4),
                             function_name='add',
                             function_value=-5)
# Tkaes around 5 minutes:
dsp((dt.now() - start).seconds)
dsp(df_params.shape)
dsp(df_params.head())

28

(172, 7)

,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
32720,IMKTA US,Q1-2022,MO_GA_SSS,"Grocery comparable store sales change, %",2.00,-3.00,2021-12-25
42440,MNRO US,Q3-2022,MO_GA_SSS,"Comparable store sales growth, %",20.00,15.00,2021-12-25
43752,ODP US,Q4-2021,MO_GA_SSS,"Retail Division Comparable Store Sales Growth, %",-3.00,-8.00,2021-12-25
10212,BOOT US,Q3-2022,MO_GA_SSS,"Same Store Sales Growth (excl. e-commerce), %",54.20,49.20,2021-12-25
40860,MCW US,Q4-2021,MO_GA_SSS,"Comparable Store Sales Growth, %",15.00,10.00,2021-12-30


In [9]:
# Pick a sample of just 4:
#sample_tickers = sample(list(df_params["ticker"].unique()), 4)
sample_tickers = list(df_params["ticker"].unique())
sample_params = df_params[df_params["ticker"].isin(sample_tickers)].copy()
len(sample_tickers)

43

# Run Scenarios on Common Time Series

In [10]:
start = dt.now()

# How much does revenue move?
return_series = "MO_RIS_REV"

# Scenarios for a sample of 10:
dicts_output=store_set.fit(sample_params,return_series)

# Print time:
dsp((dt.now() - start).seconds)

AAP US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/A9GIFJ0135/equity-models/Q3-2021.22/scenarios/RN4f6XxxV9CFGXqxHSO76Q/
ASO US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/7BX2NZ0152/equity-models/Q3-2021.22/scenarios/4gieQKYFWHe0u2OSudp-qA/
SPTN US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/LW153I0114/equity-models/Q3-2021.24/scenarios/sQ1QVsxZXGqsA0SuIFve4A/
HIBB US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/R8YJCK0168/equity-models/Q3-2022.22/scenarios/WNZQ60H5Vxy0UcF6DONk-w/
BIG US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/4AYXLX0183/equity-models/Q3-2021.21/scenarios/7nxzOX4-Ucu_7zOUJaWwaQ/
SNBR US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/0AMERM0143/equity-models/Q3-2021.24/scenarios/3nfUm0DCV6-5iNApbmJecQ/
CMG US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/8O9PZY0176/equity-models/Q3-2021.23/scenarios/Z2Bs0nwpXXCcccOV_Fux

159

In [11]:
# Show scenario results:
list_df = []
for k in dicts_output.keys():
    df = dicts_output[k].head(1)
    list_df.append(df)
pd.concat(list_df).sort_values('diff',ascending=False)

,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,SHOO US,Q4-2021,MO_RIS_REV,Net Revenue,521215541.17737,516866076.3886,0.99166
0,CVS US,Q4-2021,MO_RIS_REV,Net Revenue,72443308279.99374,71400287664.9114,0.98560
0,SPTN US,Q4-2021,MO_RIS_REV,Net Revenue,2134910743.03908,2103422681.679,0.98525
0,DECK US,Q3-2022,MO_RIS_REV,Net Revenue,1245719223.25081,1217978877.3553,0.97773
0,ODP US,Q4-2021,MO_RIS_REV,Net Revenue,2022683201.54115,1975083201.5411,0.97647
0,BOOT US,Q3-2022,MO_RIS_REV,Net Revenue,485895454.73767,472885758.9801,0.97323
0,PRTY US,Q4-2021,MO_RIS_REV,Net Revenue,698776660.25441,679878161.4089,0.97295
0,KEG-U CN,Q4-2021,MO_RIS_REV,Total Revenue,5502655.20000,5345436.48,0.97143
0,VRA US,Q4-2022,MO_RIS_REV,Net Revenue,163096458.05199,158232474.9091,0.97018
0,EXPR US,Q4-2021,MO_RIS_REV,Net Revenue,621055842.77378,599697556.5307,0.96561


In [13]:
ticker = 'SHOO US'
time_series_name = "MO_RIS_REV"
target_model_set = cd.ModelSet(ticker_list=[ticker], config=config) 
model_map = target_model_set.create_model_map(ticker=ticker,
                            time_series_name=time_series_name,
                            tree = True,
                            notebook = True,
                            col_for_labels = "time_series_name") 
model_map.show()

https://mds.canalyst.com/api/equity-model-series/II9BD40114/equity-models/Q3-2021.21/time-series/MO_RIS_REV/forecast-data-points/Q4-2021/drivers/?format=dot
